In [17]:
# Import Library Dependencies
import pandas
import matplotlib.pyplot as plt
import hvplot.pandas
import geoviews as gv

import pandas as pd
import json
import requests
from config import api_key

#country info library to get capital for coding later - pip install
from countryinfo import CountryInfo

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import Drug Consumption CSV Data
drug_consupmtion_data = pandas.read_csv("data/drug_consumption.csv")
drug_consupmtion_data.head()

,ID,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,...,Ecstasy,Heroin,Ketamine,Legal Highs,LSD,Meth,Mushrooms,Nicotine,Semer,VSA
0,1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
1,2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,...,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
2,3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,...,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
3,4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,...,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
4,5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,...,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0


In [3]:
# Identify Initial Data Information
drug_consupmtion_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1885 entries, 0 to 1884
Data columns (total 32 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              1885 non-null   int64  
 1   Age             1885 non-null   float64
 2   Gender          1885 non-null   float64
 3   Education       1885 non-null   float64
 4   Country         1885 non-null   float64
 5   Ethnicity       1885 non-null   float64
 6   Nscore          1885 non-null   float64
 7   Escore          1885 non-null   float64
 8   Oscore          1885 non-null   float64
 9   Ascore          1885 non-null   float64
 10  Cscore          1885 non-null   float64
 11  Impulsive       1885 non-null   float64
 12  SS              1885 non-null   float64
 13  Alcohol         1885 non-null   object 
 14  Amphetamines    1885 non-null   object 
 15  Amyl Nitrite    1885 non-null   object 
 16  Benzodiazepine  1885 non-null   object 
 17  Caffeine        1885 non-null   o

In [4]:
# Define the value-key pairs of the original data

data_values_definition = {
    "Age": {
        -0.95197: "18-24 years",
        -0.07854: "25-34 years",
        0.49788: "35-44 years",
        1.09449: "45-54 years",
        1.82213: "55-64 years",
        2.59171: "65+ years"},
    "Gender": {
        0.48246: "Female",
        -0.48246: "Male"},
    "Education": {
        -2.43591: "Left School Before 16 years",
        -1.73790: "Left School at 16 years",
        -1.43719: "Left School at 17 years",
        -1.22751: "Left School at 18 years",
        -0.61113: "Some College,No Certificate Or Degree",
        -0.05921: "Professional Certificate/ Diploma",
        0.45468: "University Degree",
        1.16365: "Masters Degree",
        1.98437: "Doctorate Degree"},
    "Country": {
        -0.09765: "Australia",
        0.24923: "Canada",
        -0.46841: "New Zealand",
        -0.28519: "Other",
        0.21128: "Republic of Ireland",
        0.96082: "United Kingdom",
        -0.57009: "United States"},
    "Ethnicity": {
        -0.50212: "Asian",
        -1.10702: "Black",
        1.90725: "Mixed-Black/Asian",
        0.12600: "Mixed-White/Asian",
        -0.22166: "Mixed-White/Black",
        0.11440: "Other",
        -0.31685: "White"},
    "Nscore": {-3.46436: 12,-3.15735: 13,-2.75696: 14,-2.52197: 15,-2.42317: 16,-2.3436: 17,-2.21844: 18,-2.05048: 19,-1.86962: 20,-1.69163: 21,-1.55078: 22,-1.43907: 23,-1.32828: 24,-1.1943: 25,-1.05308: 26,-0.92104: 27,-0.79151: 28,-0.67825: 29,-0.58016: 30,-0.46725: 31,-0.34799: 32,-0.24649: 33,-0.14882: 34,-0.05188: 35,0.04257: 36,0.13606: 37,0.22393: 38,0.31287: 39,0.41667: 40,0.52135: 41,0.62967: 42,0.73545: 43,0.82562: 44,0.91093: 45,1.02119: 46,1.13281: 47,1.23461: 48,1.37297: 49,1.49158: 50,1.60383: 51,1.72012: 52,1.8399: 53,1.98437: 54,2.127: 55,2.28554: 56,2.46262: 57,2.61139: 58,2.82196: 59,3.27393: 60},
    "Escore": {-3.27393: 16,-3.00537: 17,-3.00537: 18,-2.72827: 19,-2.5383: 20,-2.44904: 21,-2.32338: 22,-2.21069: 23,-2.11437: 24,-2.03972: 25,-1.92173: 26,-1.7625: 27,-1.6334: 28,-1.50796: 29,-1.37639: 30,-1.23177: 31,-1.09207: 32,-0.94779: 33,-0.80615: 34,-0.69509: 35,-0.57545: 36,-0.43999: 37,-0.30033: 38,-0.15487: 39,0.00332: 40,0.16767: 41,0.32197: 42,0.47617: 43,0.63779: 44,0.80523: 45,0.96248: 46,1.11406: 47,1.2861: 48,1.45421: 49,1.58487: 50,1.74091: 51,1.93886: 52,2.127: 53,2.32338: 54,2.57309: 55,2.8595: 56,2.8595: 57,3.00537: 58,3.27393: 59},
    "Oscore": {-3.27393: 24,-2.8595: 26,-2.63199: 28,-2.39883: 29,-2.21069: 30,-2.09015: 31,-1.97495: 32,-1.82919: 33,-1.68062: 34,-1.55521: 35,-1.42424: 36,-1.27553: 37,-1.11902: 38,-0.97631: 39,-0.84732: 40,-0.71727: 41,-0.58331: 42,-0.45174: 43,-0.31776: 44,-0.17779: 45,-0.01928: 46,0.14143: 47,0.29338: 48,0.44585: 49,0.58331: 50,0.7233: 51,0.88309: 52,1.06238: 53,1.24033: 54,1.43533: 55,1.65653: 56,1.88511: 57,1.15324: 58,2.44904: 59,2.90161: 60},
    "Ascore": {-3.46436: 12,-3.15735: 16,-3.00537: 18,-2.90161: 23,-2.78793: 24,-2.70172: 25,-2.5383: 26,-2.35413: 27,-2.21844: 28,-2.07848: 29,-1.92595: 30,-1.772: 31,-1.6209: 32,-1.47955: 33,-1.34289: 34,-1.21213: 35,-1.07533: 36,-0.91699: 37,-0.76096: 38,-0.60633: 39,-0.45321: 40,-0.30172: 41,-0.15487: 42,-0.01729: 43,0.13136: 44,0.28783: 45,0.43852: 46,0.59042: 47,0.76096: 48,0.94156: 49,1.11406: 50,1.2861: 51,1.45039: 52,1.61108: 53,1.81866: 54,2.03972: 55,2.23427: 56,2.46262: 57,2.75696: 58,3.15735: 59,3.46436: 60},
    "Cscore": {-3.46436: 17,-3.15735: 19,-2.90161: 20,-2.72827: 21,-2.57309: 22,-2.42317: 23,-2.30408: 24,-2.18109: 25,-2.04506: 26,-1.92173: 27,-1.78169: 28,-1.64101: 29,-1.5184: 30,-1.38502: 31,-1.25773: 32,-1.13788: 33,-1.0145: 34,-0.89891: 35,-0.78155: 36,-0.65253: 37,-0.52745: 38,-0.40581: 39,-0.27607: 40,-0.14277: 41,-0.00665: 42,0.12331: 43,0.25953: 44,0.41594: 45,0.58489: 46,0.7583: 47,0.93949: 48,1.13407: 49,1.30612: 50,1.46191: 51,1.63088: 52,1.81175: 53,2.04506: 54,2.33337: 55,2.63199: 56,3.00537: 57,3.46436: 59}}

drug_usage_definition = {
    "CL0": "Never Used",
    "CL1": "Used over a Decade Ago",
    "CL2": "Used in Last Decade",
    "CL3": "Used in Last Year",
    "CL4": "Used in Last Month",
    "CL5": "Used in Last Week",
    "CL6": "Used in Last Day"}

In [5]:
# Replace the Data Values of the Original DataFrame using "data_values_definition" Dictionary
drug_consupmtion_df = drug_consupmtion_data.replace(data_values_definition)
drug_consupmtion_df.head()

,ID,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,...,Ecstasy,Heroin,Ketamine,Legal Highs,LSD,Meth,Mushrooms,Nicotine,Semer,VSA
0,1,35-44 years,Female,Professional Certificate/ Diploma,United Kingdom,Mixed-White/Asian,39.0,36.0,42.0,37.0,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
1,2,25-34 years,Male,Doctorate Degree,United Kingdom,White,29.0,52.0,55.0,48.0,...,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
2,3,35-44 years,Male,Professional Certificate/ Diploma,United Kingdom,White,31.0,45.0,40.0,32.0,...,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
3,4,18-24 years,Female,Masters Degree,United Kingdom,White,34.0,34.0,46.0,47.0,...,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
4,5,35-44 years,Female,Doctorate Degree,United Kingdom,White,43.0,28.0,43.0,41.0,...,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0


In [6]:
# The Value "2.15324" on the Oscore Column was not replaced
drug_consupmtion_df["Oscore"].unique()

array([42.     , 55.     , 40.     , 46.     , 43.     , 35.     ,
       39.     , 36.     , 38.     , 47.     , 49.     , 34.     ,
       44.     , 45.     , 29.     , 51.     , 37.     , 54.     ,
       24.     , 41.     , 52.     , 48.     , 28.     , 53.     ,
       32.     , 33.     , 26.     , 31.     , 50.     ,  2.15324,
       30.     , 56.     , 57.     , 60.     , 59.     ])

In [7]:
# Remove all rows in which Oscore=2.15324
drug_consupmtion_df = drug_consupmtion_df.loc[drug_consupmtion_df["Oscore"]!=2.15324, :]
drug_consupmtion_df["Oscore"].unique()

array([42., 55., 40., 46., 43., 35., 39., 36., 38., 47., 49., 34., 44.,
       45., 29., 51., 37., 54., 24., 41., 52., 48., 28., 53., 32., 33.,
       26., 31., 50., 30., 56., 57., 60., 59.])

In [8]:
# Remove the Fictitious Drug "Semer"
drugs_to_remove = ["Semer", "Alcohol", "Caffeine", "Chocolate", "Nicotine", "Legal Highs", "Benzodiazepine"]
ilegal_drugs = ["Amphetamines", "Amyl Nitrite", "Cannabis", "Cocaine", "Crack", "Ecstasy", "Heroin", "Ketamine", "LSD", "Meth", "Mushrooms", "VSA"]
drug_consupmtion_df = drug_consupmtion_df.drop(drugs_to_remove, axis="columns")
drug_consupmtion_df = drug_consupmtion_df.loc[drug_consupmtion_df["Country"]!="Other", :]
drug_consupmtion_df

,ID,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,...,Cannabis,Cocaine,Crack,Ecstasy,Heroin,Ketamine,LSD,Meth,Mushrooms,VSA
0,1,35-44 years,Female,Professional Certificate/ Diploma,United Kingdom,Mixed-White/Asian,39.0,36.0,42.0,37.0,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0
1,2,25-34 years,Male,Doctorate Degree,United Kingdom,White,29.0,52.0,55.0,48.0,...,CL4,CL3,CL0,CL4,CL0,CL2,CL2,CL3,CL0,CL0
2,3,35-44 years,Male,Professional Certificate/ Diploma,United Kingdom,White,31.0,45.0,40.0,32.0,...,CL3,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0
3,4,18-24 years,Female,Masters Degree,United Kingdom,White,34.0,34.0,46.0,47.0,...,CL2,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0
4,5,35-44 years,Female,Doctorate Degree,United Kingdom,White,43.0,28.0,43.0,41.0,...,CL3,CL0,CL0,CL1,CL0,CL0,CL0,CL0,CL2,CL0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,1884,18-24 years,Female,"Some College,No Certificate Or Degree",United States,White,25.0,51.0,57.0,48.0,...,CL5,CL0,CL0,CL0,CL0,CL0,CL3,CL0,CL0,CL5
1881,1885,18-24 years,Male,"Some College,No Certificate Or Degree",United States,White,33.0,51.0,50.0,48.0,...,CL3,CL0,CL0,CL2,CL0,CL0,CL5,CL4,CL4,CL0
1882,1886,25-34 years,Female,University Degree,United States,White,47.0,30.0,37.0,31.0,...,CL6,CL4,CL0,CL4,CL0,CL2,CL2,CL0,CL2,CL0
1883,1887,18-24 years,Female,"Some College,No Certificate Or Degree",United States,White,45.0,26.0,48.0,32.0,...,CL6,CL0,CL0,CL3,CL0,CL0,CL3,CL0,CL3,CL0


In [9]:
# Define an empty list to fetch the countries
countries = []

# Define an empty list to fetch the data of each country
# Loop through the df to get the countries
for country in drug_consupmtion_df['Country']:
    
    # If the country is unique, then add it to our countries list
    if country not in countries:
        countries.append(country)

In [18]:
# Save config information

url = "https://api.openweathermap.org/geo/1.0/direct?"

country_data = []

# Loop through all the counties in our list to fetch country data
for country in countries:

    # get capital for more accurate lat and longitudes
    capital = CountryInfo(country).capital()

    # Create endpoint URL with each country 
    country_url = f"{url}appid={api_key}&q={capital},{country}&limit=1"
   
    try:
        country_data_response = requests.get(country_url)
        country_data_json = country_data_response.json()

        # Get the latitude and Longitude from the response
        country_latitude = country_data_json[0]['lat']
        country_longitude = country_data_json[0]['lon']
        
        # Append the country information into country_data list            
        country_data.append({"Country": country, "Capital": capital,  
                             "Lat": country_latitude, "Lon": country_longitude})

    # If an error is experienced, skip the city
    except:
        print("Country not found. Skipping...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Country not found. Skipping...
-----------------------------
Data Retrieval Complete      
-----------------------------


In [11]:
# create a DataFrame from countries, latitude, and longitude
country_data_df = pd.DataFrame(country_data)

# Show Record Count
country_data_df

,Country,Capital,Lat,Lon
0,United Kingdom,London,51.507322,-0.127647
1,Canada,Ottawa,45.420878,-75.690111
2,United States,Washington D.C.,38.895037,-77.036543
3,Australia,Canberra,-35.297591,149.101268
4,Republic of Ireland,Dublin,53.349379,-6.260559
5,New Zealand,Wellington,-41.288795,174.777211


In [12]:
#group the drug consumption by country and include the country counts
summary_df = drug_consupmtion_df.groupby('Country').size().reset_index(name='Country_Counts')

#merge the summary_df and country_data_df on country
merged_df = pd.merge(summary_df, country_data_df, on='Country')

merged_df

,Country,Country_Counts,Capital,Lat,Lon
0,Australia,54,Canberra,-35.297591,149.101268
1,Canada,84,Ottawa,45.420878,-75.690111
2,New Zealand,5,Wellington,-41.288795,174.777211
3,Republic of Ireland,20,Dublin,53.349379,-6.260559
4,United Kingdom,1040,London,51.507322,-0.127647
5,United States,546,Washington D.C.,38.895037,-77.036543


In [13]:
#go through each row of the drug_consumption_df, checking the data for all drug columns assign 0 if value is CL0, otherwise assign 1
#put the value in the Drugs column
#Used ChatGPT to ask Python code to loop through a dataset on one column and then loop across 9 columns 
#for each row to check if value = 'CL0' and create new column to set to 1 if true or 0 if false

drug_consupmtion_df['Drugs'] = drug_consupmtion_df.apply(lambda row: 1 if row['Crack'] != "CL0" 
                                                         or row['Cocaine'] != "CL0" 
                                                         or row['Ecstasy'] != "CL0" 
                                                         or row['Heroin'] != "CL0" 
                                                         or row['Ketamine'] != "CL0" 
                                                         or row['LSD'] != "CL0" 
                                                         or row['Meth'] != "CL0" 
                                                         or row['Mushrooms'] != "CL0" 
                                                         or row['VSA'] != "CL0" 
                                                         else 0, axis=1)
drug_consupmtion_df

,ID,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,...,Cocaine,Crack,Ecstasy,Heroin,Ketamine,LSD,Meth,Mushrooms,VSA,Drugs
0,1,35-44 years,Female,Professional Certificate/ Diploma,United Kingdom,Mixed-White/Asian,39.0,36.0,42.0,37.0,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,0
1,2,25-34 years,Male,Doctorate Degree,United Kingdom,White,29.0,52.0,55.0,48.0,...,CL3,CL0,CL4,CL0,CL2,CL2,CL3,CL0,CL0,1
2,3,35-44 years,Male,Professional Certificate/ Diploma,United Kingdom,White,31.0,45.0,40.0,32.0,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,1
3,4,18-24 years,Female,Masters Degree,United Kingdom,White,34.0,34.0,46.0,47.0,...,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,1
4,5,35-44 years,Female,Doctorate Degree,United Kingdom,White,43.0,28.0,43.0,41.0,...,CL0,CL0,CL1,CL0,CL0,CL0,CL0,CL2,CL0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880,1884,18-24 years,Female,"Some College,No Certificate Or Degree",United States,White,25.0,51.0,57.0,48.0,...,CL0,CL0,CL0,CL0,CL0,CL3,CL0,CL0,CL5,1
1881,1885,18-24 years,Male,"Some College,No Certificate Or Degree",United States,White,33.0,51.0,50.0,48.0,...,CL0,CL0,CL2,CL0,CL0,CL5,CL4,CL4,CL0,1
1882,1886,25-34 years,Female,University Degree,United States,White,47.0,30.0,37.0,31.0,...,CL4,CL0,CL4,CL0,CL2,CL2,CL0,CL2,CL0,1
1883,1887,18-24 years,Female,"Some College,No Certificate Or Degree",United States,White,45.0,26.0,48.0,32.0,...,CL0,CL0,CL3,CL0,CL0,CL3,CL0,CL3,CL0,1


In [14]:
#sum the drug consumption for each country
sum_drug_consupmtion_df = drug_consupmtion_df.groupby("Country")['Drugs'].sum().reset_index()
sum_drug_consupmtion_df

,Country,Drugs
0,Australia,45
1,Canada,66
2,New Zealand,5
3,Republic of Ireland,16
4,United Kingdom,497
5,United States,514


In [15]:
#merge the merged_df and the sum_drug_consumption_df on country
prop_merged_df = pd.merge(merged_df, sum_drug_consupmtion_df, on='Country')

#Calculate the percentage drug use for each country
prop_merged_df['Percent_drug_use'] = prop_merged_df.apply(lambda row: (row['Drugs']/row['Country_Counts'])*100, axis=1)

prop_merged_df

,Country,Country_Counts,Capital,Lat,Lon,Drugs,Percent_drug_use
0,Australia,54,Canberra,-35.297591,149.101268,45,83.333333
1,Canada,84,Ottawa,45.420878,-75.690111,66,78.571429
2,New Zealand,5,Wellington,-41.288795,174.777211,5,100.000000
3,Republic of Ireland,20,Dublin,53.349379,-6.260559,16,80.000000
4,United Kingdom,1040,London,51.507322,-0.127647,497,47.788462
5,United States,546,Washington D.C.,38.895037,-77.036543,514,94.139194


In [16]:
# Configure the map plot
map_plot = prop_merged_df.hvplot.points(
    "Lon",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = "Percent_drug_use",
    scale = 3.0,
    color = "Drugs",
    legend=True,
    hover_cols=['Country']
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lon,Lat]   (Drugs,Percent_drug_use,Country)